# Receipts Auto Form Filling

Clone the repo and change the system directory to the cloned repo.

In [1]:
!git clone https://gitlab.com/gyan42/receipts-form-filling

Cloning into 'receipts-form-filling'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 92 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [2]:
% cd receipts-form-filling/

/content/receipts-form-filling


In [3]:
! git pull

Already up to date.


Install all the required Python packages

In [4]:
!pip install -r colab-requirements.txt

     |████████████████████████████████| 221 kB 5.3 MB/s 
     |████████████████████████████████| 2.1 MB 36.1 MB/s 
     |████████████████████████████████| 18.1 MB 46 kB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 69 kB 6.2 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 243 kB 48.3 MB/s 
     |████████████████████████████████| 132 kB 47.0 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 3.3 MB 37.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=8179cfff4add730665c98a3e06e6fca393729e8c53d73d792bdf844b3ea1165c
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.

In [5]:
!pwd

/content/receipts-form-filling


In [6]:
!ls

api			configs    ops	      requirements.txt
colab-requirements.txt	ner-model  README.md  ui


Add repo Python sourch to system Python path 

In [7]:
% set_env PYTHONPATH="/env/python:/content/receipts-form-filling/ner-model/src"

env: PYTHONPATH="/env/python:/content/receipts-form-filling/ner-model/src"


Check the torch version on Colab.

In [8]:
# List all GPUs
import torch
"Torch version:" + "\"" + torch.__version__  + "\"" + " and current GPU allocated is "  + "\"" + torch.cuda.get_device_name(torch.cuda.current_device()) + "\""

'Torch version:"1.10.0+cu111" and current GPU allocated is "Tesla K80"'

In [9]:
! ls /content/receipts-form-filling/ner-model/src

gyan42


In [10]:
! echo $PYTHONPATH # /env/python

"/env/python:/content/receipts-form-filling/ner-model/src"


In [11]:
% cd ner-model/src/

/content/receipts-form-filling/ner-model/src


## Model Training

In [12]:
import numpy as np
import transformers
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset, load_metric
# Ref: https://github.com/huggingface/datasets/blob/master/datasets/conll2003/conll2003.py
import os
from pathlib import Path

import datasets
from datasets import DownloadConfig
import os
from pathlib import Path
from datasets import load_dataset, ClassLabel, DownloadConfig

from transformers import AutoTokenizer
from gyan42.dataset.hf_tokenize import HFTokenizer
from gyan42.dataset.sroie2019_dataset import HFSREIO2019Dataset

In [13]:
metric = load_metric("seqeval")

logger = datasets.logging.get_logger(__name__)

In [14]:
def compute_metrics(p, label_list):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [32]:
model_n_version = "sroie2019v1"
max_epochs = 15
learning_rate = 2e-5
batch_size = 16
model_root_dir = "~/.gyan42/models/hf/"

hf_pretrained_model_checkpoint = "distilbert-base-uncased"
hf_pretrained_tokenizer_checkpoint = "distilbert-base-uncased"

hf_dataset = HFSREIO2019Dataset()
hf_preprocessor = HFTokenizer.init_vf(hf_pretrained_tokenizer_checkpoint=hf_pretrained_tokenizer_checkpoint)

hf_model = AutoModelForTokenClassification.from_pretrained(hf_pretrained_model_checkpoint,
                                                        num_labels=len(hf_dataset.labels))

hf_model.config.id2label = hf_dataset.id2label
hf_model.config.label2id = hf_dataset.label2id

tokenized_datasets = hf_dataset.dataset.map(hf_preprocessor.tokenize_and_align_labels, batched=True)

Reusing dataset sroi_e2019 (/root/.mozhi/sroi_e2019/SROIE2019/1.0.0)


Cache directory:  /root/.mozhi
Cache1 directory:  /root/.mozhi/sroi_e2019/SROIE2019/1.0.0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

Loading cached processed dataset at /root/.mozhi/sroi_e2019/SROIE2019/1.0.0/cache-9face021980ae63f.arrow


In [33]:

# ---------------------------------------------------------------------------------------------------

args = TrainingArguments(
    f"test-ner",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=max_epochs,
    weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(hf_preprocessor.tokenizer)

trainer = Trainer(
    hf_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=hf_preprocessor.tokenizer,
    compute_metrics=lambda p: compute_metrics(p=p, label_list=hf_dataset.labels)
)

trainer.train()
trainer.evaluate()

# Predictions on test dataset and evaluation

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [hf_dataset.labels[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [hf_dataset.labels[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

out_dir = os.path.expanduser(model_root_dir) + "/" + model_n_version
trainer.save_model(out_dir)

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: company seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: address seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: total seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: date seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

{'ate': {'precision': 0.6909090909090909, 'recall': 0.7378640776699029, 'f1': 0.7136150234741784, 'number': 103}, 'ddress': {'precision': 0.8351648351648352, 'recall': 0.8444444444444444, 'f1': 0.839779005524862, 'number': 90}, 'ompany': {'precision': 0.6835443037974683, 'recall': 0.6666666666666666, 'f1': 0.675, 'number': 81}, 'otal': {'precision': 0.5384615384615384, 'recall': 0.5957446808510638, 'f1': 0.5656565656565657, 'number': 94}, 'overall_precision': 0.6822916666666666, 'overall_recall': 0.7119565217391305, 'overall_f1': 0.696808510638298, 'overall_accuracy': 0.9702717830710865}


In [34]:
! ls ~/.gyan42/models/hf/sroie2019v1/

config.json	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer_config.json    vocab.txt


## Model Packaging for Serving

In [35]:
%set_env SERIALIZED_MODEL_FILE=/root/.gyan42/models/hf/sroie2019v1/pytorch_model.bin

env: SERIALIZED_MODEL_FILE=/root/.gyan42/models/hf/sroie2019v1/pytorch_model.bin


In [36]:
! mkdir -p  ${HOME}/.gyan42/model-store/

In [37]:
!torch-model-archiver --force \
--model-name sroie2019v1 \
--version 1.0 \
--serialized-file  $SERIALIZED_MODEL_FILE \
--handler gyan42/serving/handler/hf_transformer_handler.py \
--extra-files /root/.gyan42/models/hf/sroie2019v1/config.json,/root/.gyan42/models/hf/sroie2019v1/special_tokens_map.json,/root/.gyan42/models/hf/sroie2019v1/training_args.bin,/root/.gyan42/models/hf/sroie2019v1/tokenizer_config.json,/root/.gyan42/models/hf/sroie2019v1/vocab.txt \
--export-path /root/.gyan42/model-store/

WARNING - Overwriting /root/.gyan42/model-store/sroie2019v1.mar ...


In [38]:
! ls /root/.gyan42/model-store/

sroie2019v1.mar


In [39]:
! cp /root/.gyan42/model-store/sroie2019v1.mar  /content/receipts-form-filling/

In [40]:
from google.colab import files
files.download('/content/receipts-form-filling/sroie2019v1.mar')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>